In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset = pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent = dataset[['age', 'bmi', 'children','sex_male','smoker_yes']]

In [8]:
dependent = dataset[["charges"]]

In [9]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
x_train

array([[-1.5330973 , -0.40713453, -0.89833872, -0.97676557, -0.50466988],
       [-0.03364163,  0.32855417, -0.89833872, -0.97676557, -0.50466988],
       [ 0.89459283,  2.56690911,  3.25603402, -0.97676557, -0.50466988],
       ...,
       [ 0.03776102, -0.91016269, -0.89833872,  1.02378711, -0.50466988],
       [-1.46169465,  0.76659782, -0.89833872,  1.02378711, -0.50466988],
       [-0.46205754, -1.96596021, -0.06746417, -0.97676557, -0.50466988]])

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_grid = {'kernel':['linear','rbf','poly','sigmoid'],'C':[10,100,500,1000,2000,3000,5000,10000],'gamma':['auto','scale']}
grid = GridSearchCV(SVR(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


C:\Users\Genuine\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [10, 100, 500, 1000, 2000, 3000, 5000, 10000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test)

from sklearn.metrics import r2_score
r_score = r2_score(y_test,grid_predictions)
print('The R_score value for best parameter {}:'.format(grid.best_params_),r_score)

The R_score value for best parameter {'C': 10000, 'gamma': 'scale', 'kernel': 'rbf'}: 0.8779952426221569


In [14]:
table = pd.DataFrame.from_dict(re)

In [15]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.144124,0.087534,0.015626,2.582856e-06,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,45
1,0.087502,0.007656,0.040629,7.658296e-03,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.004055,0.013366,-0.103821,-0.095119,-0.101604,-0.056625,0.053504,55
2,0.062503,0.000002,0.021879,7.657245e-03,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.056274,0.069532,-0.045601,-0.025079,-0.049592,0.001107,0.051309,52
3,0.106254,0.011695,0.025005,7.656466e-03,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.049905,0.075905,-0.046585,-0.041004,-0.046507,-0.001657,0.053391,54
4,0.075005,0.006254,0.012503,6.251574e-03,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.112503,0.006249,0.018755,6.250930e-03,10000,auto,sigmoid,"{'C': 10000, 'gamma': 'auto', 'kernel': 'sigmo...",-27.033602,-29.729510,-21.407864,-31.142464,-20.607168,-25.984122,4.279991,63
60,0.294025,0.028390,0.015631,1.934572e-06,10000,scale,linear,"{'C': 10000, 'gamma': 'scale', 'kernel': 'line...",0.660813,0.581368,0.697919,0.528624,0.655731,0.624891,0.061215,25
61,0.256262,0.028984,0.037503,7.657557e-03,10000,scale,rbf,"{'C': 10000, 'gamma': 'scale', 'kernel': 'rbf'}",0.838113,0.806986,0.828970,0.824149,0.752120,0.810068,0.030691,1
62,1.018865,0.242996,0.015630,6.503192e-07,10000,scale,poly,"{'C': 10000, 'gamma': 'scale', 'kernel': 'poly'}",0.849881,0.803719,0.817393,0.804084,0.763612,0.807738,0.027734,3


In [16]:
age_input=int(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=int(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Somker Yes 0 or 1:"))

Age:33
BMI:255.68
Children:3
Sex Male 0 or 1:1
Somker Yes 0 or 1:0


In [17]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print('Future_Prediction = {}'.format(Future_Prediction))

Future_Prediction = [17690.41791398]
